In [98]:
import os
import requests
import queue
import pandas as pd
import threading
from bs4 import BeautifulSoup
from tqdm import tqdm

DATA_PATH = "../data"
DICTS_PATH = "../dicts"
DATA_FILENAME = "valid.csv"

data_file = os.path.join(DATA_PATH, DATA_FILENAME)

In [99]:
valid_df = pd.read_csv(os.path.join(data_file), index_col=0, dtype={"purch": str, "winer": str})

In [100]:
valid_df.shape

(23366, 28)

In [101]:
headers = {'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
           'sec-ch-ua-mobile':'0',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }

def get_winer_info_vie_purch(purch, q):   
    url = "https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber=" + str(purch) 
    r = requests.get(url, headers = headers)
    text = r.content.decode('utf-8')
    soup = BeautifulSoup(text)
    winer_list = soup.find_all('table', {'class': 'blockInfo__table tableBlock'})
    if len(winer_list) > 3:
        soup = BeautifulSoup(str(winer_list[3]))
    elif len(winer_list) == 3:
        soup = BeautifulSoup(str(winer_list[2]))
    contract_link = soup.find('a')
    contract_url = contract_link.get('href')
    contract_url = 'https://zakupki.gov.ru' + contract_url
    contract_page = requests.get(contract_url, headers = headers)
    contract_page_text = contract_page.content.decode('utf-8')
    soup = BeautifulSoup(contract_page_text)
    # Find winer info
    info = soup.find_all("tbody", {'class': "tableBlock__body"})[0]
    info_dict = {"inn": "","name": "", "phone": "", "address": ""}
    for i in range(len(info.find_all("td"))):
        if i == 0:
            info_dict["name"] = info.find_all("td")[i].text.split('\n')[1]
        if i == 2:
            info_dict["address"] = info.find_all("td")[i].text
        if i == 4:
            info_dict["phone"] = info.find_all("td")[i].text
    # Find winer inn
    spans = soup.find_all('span')
    ind_cur = -1
    for span in spans:
        if span.text == 'ИНН:': 
            ind_cur = spans.index(span) + 1
            break
    info_dict["inn"] = spans[ind_cur].text
    
    q.put(info_dict)
    return
tqdm.pandas()

In [102]:
def start_download(purch):
    out = purch
    q = queue.Queue()
    scraping = threading.Thread(target=get_winer_info_vie_purch, args=(out,q,))
    scraping.start()
    scraping.join()
    while threading.active_count() > 15:
        time.sleep(0.01)
    return q.get()

In [103]:
inns = {'0224013045',
 '0323037270',
 '0326040626',
 '1101107584',
 '1101145043',
 '1328013394',
 '1435090876',
 '1649036481',
 '1655303424',
 '1656033379',
 '1660240931',
 '1660328953',
 '1901119814',
 '1901120601',
 '2130209293',
 '2221005804',
 '2222798183',
 '2224121968',
 '2225181208',
 '230203093245',
 '2312271360',
 '2460002949',
 '2461019543',
 '2464019742',
 '2465059970',
 '2465091580',
 '2465320039',
 '2466055150',
 '2466169528',
 '2466189073',
 '2466272518',
 '2540055907',
 '2540087761',
 '2540221551',
 '2543079825',
 '2628058630',
 '2635207403',
 '2801252525',
 '2807032170',
 '3025003760',
 '3120099928',
 '3123321867',
 '3308003627',
 '3327110751',
 '3443923035',
 '352532976647',
 '3525357197',
 '3662195283',
 '3662212115',
 '3662992257',
 '366300529071',
 '3702028556',
 '3703022973',
 '3711026548',
 '3808084790',
 '391300326265',
 '420544385832',
 '4209005047',
 '4253003592',
 '434200002901',
 '4345468090',
 '4347029500',
 '4501204468',
 '4632197133',
 '5005066205',
 '5017083029',
 '5032294384',
 '5035031009',
 '5038033276',
 '5041016087',
 '5047166032',
 '5074039026',
 '5249154390',
 '5252042135',
 '5252045070',
 '5260200603',
 '5262258091',
 '5402546938',
 '5404041675',
 '5404250100',
 '5404405629',
 '5405161372',
 '5407211477',
 '5408152658',
 '5408291034',
 '5409241861',
 '5501072090',
 '5506226202',
 '5507204882',
 '5526006170',
 '5611053132',
 '5612066367',
 '561605887857',
 '5720022381',
 '575201465797',
 '5754004670',
 '5902050496',
 '590400018275',
 '5904231307',
 '5908051626',
 '5916020413',
 '6143083623',
 '6150067809',
 '615520705111',
 '6163109990',
 '6165121914',
 '6165181247',
 '6165196363',
 '616606649213',
 '6312125239',
 '6312171877',
 '6314018352',
 '6321070900',
 '6321414100',
 '6448012000',
 '6449078860',
 '6454016806',
 '6455018450',
 '645505818053',
 '6658344555',
 '6658448949',
 '6663071261',
 '6678028357',
 '6685071025',
 '6686059581',
 '6730018851',
 '6732027668',
 '6901046523',
 '6950223535',
 '7017341019',
 '7017350373',
 '7017398456',
 '7017456010',
 '7202251472',
 '7203241685',
 '7204032814',
 '7204112210',
 '7204205993',
 '7325140324',
 '7430018884',
 '7449063094',
 '7449088941',
 '7451344670',
 '7453271523',
 '753600159769',
 '7702019044',
 '7703406695',
 '7704413624',
 '7704543951',
 '7705042179',
 '7705952210',
 '7707049388',
 '7707086510',
 '7709487997',
 '7709753021',
 '7710316675',
 '7713447235',
 '7714072839',
 '7714939790',
 '7715768700',
 '7716017505',
 '7716644263',
 '7716814525',
 '7716874274',
 '7717764683',
 '7718599175',
 '7719481754',
 '7722715321',
 '7723322154',
 '7723378990',
 '7725285067',
 '7725375730',
 '7726362290',
 '7726414773',
 '7726448684',
 '7727173908',
 '7729079361',
 '7729418511',
 '7730702372',
 '7731329393',
 '7731416945',
 '7733865216',
 '7740000076',
 '7751154455',
 '7801557131',
 '7802411953',
 '7802744800',
 '7802829934',
 '7804372516',
 '7805074390',
 '7806005744',
 '781018594156',
 '7813257774',
 '7816530372',
 '7816598363',
 '7842032486',
 '8601023568',
 '8602212536',
 '860604211188',
 '8617035150',
 '9701120430',
 '9701146325',
 '9731038990'}

In [104]:
df['info'] = df['purch'].progress_apply(start_download)

  8%|███████▌                                                                                  | 17/203 [00:13<02:32,  1.22it/s]


KeyboardInterrupt: 

In [76]:
df = pd.DataFrame(columns=valid_df.columns)

In [93]:
for i in range(valid_df.shape[0]):
    if valid_df.iloc[i].winer in inns:
        df = df.append(valid_df.iloc[i])
        inns.remove(valid_df.iloc[i].winer)
        continue

In [96]:
df.info

<bound method DataFrame.info of        Закупки по Реестровый номер закупки  \
14710       44-ФЗ     №0360300029118000043   
20476       44-ФЗ     №0156200009919000010   
20600       44-ФЗ     №0142300029019000008   
21117       44-ФЗ     №0373200041519000081   
21154       44-ФЗ     №0891200000619000096   
...           ...                      ...   
96100       44-ФЗ     №0817200000321003183   
120911      44-ФЗ     №0860200000819006199   
122948      44-ФЗ     №0372100008620000044   
130231      44-ФЗ     №0139200000121004603   
187291      44-ФЗ     №0387200017719000162   

       Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)  \
14710                                 Электронный аукцион                                                    
20476                Открытый конкурс в электронной форме                                                    
20600                                 Электронный аукцион                            

In [97]:
df["info"].to_csv("../data/inn_info.csv", index=False)

In [110]:
df["inn"] = df["info"].apply(lambda x: x["inn"])

In [113]:
df["company_name"] = df["info"].apply(lambda x: x["name"])

In [118]:
df["address"] = df["info"].apply(lambda x: x["address"])
df["address"] = df["address"].apply(lambda s: s.strip())

In [126]:
df["phone"] = df["info"].apply(lambda x: x["phone"])
df["phone"] = df["phone"].apply(lambda s: s.strip())

In [129]:
valid_df = 

14710      6454016806
20476      7707049388
20600      3120099928
21117      3327110751
21154      7802829934
21482      7451344670
21580      6686059581
21763      5612066367
21866      5904231307
22090      7714072839
22108      6314018352
22126      5038033276
22158      6165121914
22214      5047166032
22263      7725285067
22294      1101107584
22402      7449088941
22420      6321070900
22464      6732027668
22681      3662992257
22873      7806005744
22879      2312271360
22908      7727173908
22986      7716644263
23073      7715768700
23120      7718599175
23226      6165196363
23424      5408291034
23431      5041016087
23476      7453271523
23489      7202251472
23743      2540087761
23907      7449063094
24012      6163109990
24024      8601023568
24035      2628058630
24188    781018594156
24207    753600159769
24228      5074039026
24257    420544385832
24331      7204112210
24384      7804372516
24392      3308003627
24916      3525357197
24917      5754004670
24940     